In [ ]:
import pandas as pd
import numpy as np
import pyam

import matplotlib.pyplot as plt
from matplotlib import cm
import mpl_toolkits.mplot3d.axes3d as axes3d

import ixmp
import message_ix

mp = ixmp.Platform(name="default", jvmargs=["-Xmx12G"])

from message_data.tools.utilities import get_nodes

In [ ]:
1.0 / 0.886377678483484

In [ ]:
# Converts US2000$ to US2005$
usd_conv_2000_2005 = 1.0 / 0.886377678483484

# Converts US2000$ to US2010$
usd_conv_2000_2010 = 1.0 / 0.886377678483484 * 1.10774

# Converts US2005$ to US2010$
usd_conv_2005_2010 = 1.10774


def retrieve_ghgcat(land_scen):
    # Filter out the amount of carbon price categories
    # The names of the prices are based on the globiom matrix, hence these need to be converted from
    # 2000$ -> 2010$
    ghg_cat = (
        pd.DataFrame.from_dict(
            {
                "GHG" + x.split("GHG")[1]: round(
                    float(x.split("GHG")[1]) * usd_conv_2000_2010, 2
                )
                for x in land_scen
            },
            orient="index",
        )
        .rename(columns={0: "GHG"})
        .sort_values(by="GHG")
    )
    Xs = np.array(ghg_cat["GHG"])
    return (Xs, ghg_cat)


def retrieve_bio(scen, region, land_scen):
    bio_base_cat = sorted(
        list(set([x for x in land_scen if "GHG000" in x and "N" not in x]))
    )
    df = scen.par(
        "land_output",
        filters={"level": ["land_use"], "land_scenario": bio_base_cat, "node": region},
    )
    df = df[(df["year"] >= ymin) & (df["year"] <= ymax)]
    years = df["year"].unique()
    df = df.set_index("year")

    # Retrieve duration period, multiply values and sum across all regions
    df["duration"] = scen.par("duration_period", filters={"year": years}).set_index(
        "year"
    )["value"]
    df["value"] *= df["duration"] / 1000 / 31.71
    df = (
        df.reset_index()
        .groupby(["land_scenario"])
        .sum()
        .drop(["duration", "year"], axis=1)
    )
    df["value"] = round(df.value, 2)
    Xy = np.array(df.value)
    return (Xy, bio_base_cat)


def retrieve_TCE(scen, region, ghg_cat):
    df = scen.par("land_emission", filters={"emission": ["TCE"], "node": region})
    df = df[(df["year"] >= ymin) & (df["year"] <= ymax)]
    years = df["year"].unique()
    df = df.set_index("year")

    # Retrieve duration period, multiply values and sum across all regions
    df["duration"] = scen.par("duration_period", filters={"year": years}).set_index(
        "year"
    )["value"]
    df["value"] *= df["duration"] / 1000 / (12 / 44)
    df = (
        df.reset_index()
        .groupby(["land_scenario"])
        .sum()
        .drop(["duration", "year"], axis=1)
    )
    df["value"] = round(df.value, 2)

    df = df.reset_index()
    df["bio_cat"] = df.land_scenario.apply(lambda x: x.split("GHG")[0])
    df["ghg_cat"] = df.land_scenario.apply(lambda x: "GHG" + x.split("GHG")[1])
    df = df.drop("land_scenario", axis=1).pivot_table(
        index="bio_cat", columns="ghg_cat", values="value"
    )
    df = df[ghg_cat.index]
    Zs = np.array(df)
    return Zs


def retrieve_results(scen, regions, ghg_cat, Xs, Ys, Zs):
    df = scen.var("LAND", filters={"node": regions})
    df = df[(df["year"] >= ymin) & (df["year"] <= ymax)]
    df = df[df["land_scenario"].str.find("N") < 0]
    df = (
        df.reset_index()
        .groupby(["land_scenario"])
        .sum()
        .drop(["year", "index", "mrg"], axis=1)
    )
    df["lvl"] = round(df.lvl, 2)
    df = df.reset_index()
    df["bio_cat"] = df.land_scenario.apply(lambda x: x.split("GHG")[0])
    df["ghg_cat"] = df.land_scenario.apply(lambda x: "GHG" + x.split("GHG")[1])
    df = df.drop("land_scenario", axis=1).pivot_table(
        index="bio_cat", columns="ghg_cat", values="lvl"
    )
    df = df[ghg_cat.index]
    df = df.replace(0, np.nan)
    Zs_res = np.array(df)
    Zs_res_tmp = np.array(
        df.replace([x for x in np.unique(df.values) if not np.isnan(x)], 1)
    )
    Zs_res = Zs * Zs_res_tmp
    Xs_res = Xs * Zs_res_tmp
    Ys_res = Ys * Zs_res_tmp
    return (Xs_res, Ys_res, Zs_res)

### Figure 1. Create Land-Use Surface figure

In [ ]:
def lu_surface(
    scenarios, region="all", plot_results=False, title="main", titles=[], rows=1
):
    for model_nm in scenarios:
        if title == "main":
            figsize = (12, 8)
            font_ax_title = 15
            font_ax_label = 13
            font_ax_tick = 11
            font_title = 16
            dist = 13
            labelpad = 25
        elif title == "scenario":
            figsize = (20, 14)
            font_ax_title = 17
            font_ax_label = 15
            font_ax_tick = 13
            font_title = 22
            dist = 10
            labelpad = 25

        fig = plt.figure(figsize=figsize)
        mod_list = ["a.", "b.", "c.", "d."]
        for i, scen_nm in enumerate(scenarios[model_nm]):
            # Load scenario
            scen = message_ix.Scenario(mp, model_nm, scen_nm)

            if region == "all":
                regs = get_nodes(scen)
            else:
                regs = region

            # Retrieve list of land scenarios
            land_scen = scen.set("land_scenario")

            # Create X-array, which will contain all the GHG price categories
            Xs, ghg_cat = retrieve_ghgcat(land_scen)

            # Create Y-array, which contains all the cumulative biomass potentials, by category
            Ys, bio_base_cat = retrieve_bio(scen, regs, land_scen)

            Xs, Ys = np.meshgrid(Xs, Ys)

            # Create Z-array, which containts the cumulative GHG emissions per biomass category
            Zs = retrieve_TCE(scen, regs, ghg_cat)

            if plot_results:
                Xs_res, Ys_res, Zs_res = retrieve_results(
                    scen, regs, ghg_cat, Xs, Ys, Zs
                )

            # Set position
            cols = int(len(scenarios[model_nm]) / rows)
            index = scenarios[model_nm].index(scen_nm) + 1

            ax = fig.add_subplot(rows, cols, index, projection="3d")

            # plot wireframe
            ax.plot_wireframe(Xs, Ys, Zs, rstride=4, cstride=4, alpha=0.6)

            # plot surface, more colormaps: http://matplotlib.sourceforge.net/examples/pylab_examples/show_colormaps.html
            ax.plot_surface(Xs, Ys, Zs, rstride=4, cstride=4, alpha=0.6)

            if plot_results:
                ax.plot_surface(Xs_res, Ys_res, Zs_res, rstride=4, cstride=4, alpha=0.8)

            if title == "scenario":
                ax.set_title(titles[index - 1], fontsize=font_ax_title, y=0.99)
                ax.text(x=0, y=0, z=1700, s=mod_list[i], fontsize=18)
            ax.xaxis.set_rotate_label(False)
            ax.set_xlabel("Carbon Price\nin [$2010/tCO2]", fontsize=font_ax_label)
            ax.yaxis.set_rotate_label(False)
            ax.set_ylabel(
                "Biomass potential\nin [GJ]", fontsize=font_ax_label, rotation=50
            )
            ax.zaxis.set_rotate_label(False)
            ax.set_zlabel(
                "GHG Emissions\nin [GtCO2e]", fontsize=font_ax_label, rotation=90
            )
            ax.dist = dist
            ax.xaxis.labelpad = labelpad
            ax.yaxis.labelpad = labelpad
            ax.zaxis.labelpad = labelpad
            ax.tick_params(axis="both", which="major", labelsize=font_ax_tick)
            # ax.set_frame_on(True)

        if region == "all":
            reg_title = "Global"
        else:
            reg_title = region[0]

        if title == "main":
            plt.suptitle(
                "{} Land-use Tradeoff Surface\nCumulated from {} to {}".format(
                    reg_title, ymin, ymax
                ),
                fontsize=font_title,
                y=0.85,
            )
        elif title == "scenario":
            plt.suptitle(
                "{} Land-use Tradeoff Surface incl. results\nCumulated from {} to {}".format(
                    reg_title, ymin, ymax
                ),
                fontsize=font_title,
            )

        plt.tight_layout()

        if plot_results:
            plt.savefig(
                "{}_{}_LanduseSurface_incl_results.png".format(model_nm, reg_title),
                bbox_inches="tight",
                transparent=True,
            )
        else:
            plt.savefig(
                "{}_{}_LanduseSurface.png".format(model_nm, reg_title),
                bbox_inches="tight",
                transparent=True,
            )

### Figure 2. Create Land-Use Matrix

In [ ]:
def lu_matrix(scen):
    # Retrieve list of land scenarios
    land_scen = scen.set("land_scenario")

    Xs, ghg_cat = retrieve_ghgcat(land_scen)

    bio_cat = (
        pd.DataFrame.from_dict(
            {
                x.split("GHG")[0]: int(x.split("GHG")[0].replace("BIO", ""))
                for x in land_scen
                if "N" not in x
            },
            orient="index",
        )
        .rename(columns={0: "BIO"})
        .sort_values(by="BIO")
    )
    # Manual addition of biomass prices
    # The names of the prices are based on the globiom matrix, hence these need to be converted from
    # 2000$ -> 2010$
    bio_cat["Price in US$2010/GJ"] = [
        round(p * usd_conv_2000_2010, 2) for p in [1.0, 1.5, 3.0, 5.0, 8.0, 13.5, 40.0]
    ]

    _data = []
    c = 0
    for i in range(len(bio_cat.index)):
        _data.append([i + c for i in range(len(ghg_cat.index))])
        c += 1
    # Setting up multi-panel figure
    # Create plot
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot()

    im = ax.imshow(_data, cmap="Greens")

    ax.xaxis.tick_top()
    ## We want to show all ticks...
    ax.set_xticks(np.arange(len(ghg_cat.index)))
    ax.set_yticks(np.arange(len(bio_cat.index)))
    ## ... and label them with the respective list entries
    ax.set_xticklabels(
        ["{}\n({})".format(x, ghg_cat.loc[x, "GHG"]) for x in ghg_cat.index]
    )
    ax.set_yticklabels(
        [
            "{}\n({})".format(x, bio_cat.loc[x, "Price in US$2010/GJ"])
            for x in bio_cat.index
        ]
    )
    ax.set_xlabel("Carbon Price Categories\n(Price in[$2010/tCO2])", fontsize=16)
    ax.set_ylabel("Biomass Price Categories\n(Price in [$2010/GJ])", fontsize=16)
    ax.xaxis.labelpad = -530
    ax.yaxis.labelpad = 30

    ## Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), ha="center")

    title = "Land-Use Pathway Scenario Matrix"
    ax.set_title("{}\n".format(title), fontsize=21)

    plt.savefig("{}.png".format("_".join(title.split(" "))), bbox_inches="tight")

### Figure 3. Create Land-Use by land_type comparison

In [ ]:
def lu_developments(scen, region, land_scen):
    nrows = int(len(land_scen) / 2)
    ncols = int(len(land_scen) / 2)
    fig, axs = plt.subplots(
        ncols, nrows, figsize=(6 * nrows, 8), sharex=True, sharey=True
    )

    plotted_ls = []
    cnt = 1
    for axis in axs:
        for ls, ax in zip([ls for ls in land_scen if ls not in plotted_ls], axis):
            kwargs = {"legend": False, "title": ls}
            plotted_ls.append(ls)
            df = (
                scen.par("land_use", filters={"node": [region], "land_scenario": [ls]})
                .drop(["node", "land_scenario", "unit"], axis=1)
                .pivot_table(index="year", columns="land_type", values="value")
                .rename(
                    columns={
                        "CrpLnd": "Crop Land",
                        "GrsLnd": "Grass Land",
                        "NewFor_G4M": "New Forest",
                        "OldFor_G4M": "Old Forest",
                        "OtherLnd": "Other Land",
                        "PltFor": "Plantation Forest",
                    }
                )
            )
            df = df[
                [
                    "Crop Land",
                    "Other Land",
                    "Grass Land",
                    "New Forest",
                    "Plantation Forest",
                    "Old Forest",
                ]
            ]
            df *= 100
            df.plot.area(
                ax=ax,
                **kwargs,
                color=["darkgreen", "grey", "lightgreen", "chocolate", "gold", "brown"],
            )
            if ls == land_scen[-1]:
                ax.legend()
                handles, labels = ax.get_legend_handles_labels()
                # sort both labels and handles by labels
                labels.reverse()
                handles.reverse()
                ax.legend(
                    handles,
                    labels,
                    loc="lower right",
                    bbox_to_anchor=(1.6, 0),
                    shadow=True,
                    title="Land Types",
                    title_fontsize=12,
                )

            ax.set_xlim(min(df.index), 2100)

            ax.set_ylabel("Land types by share in [%]")

            cnt += 1

    # Add labels
    title = "Land-use across different Land-use Pathways for {}".format(region)
    fig.suptitle(title, fontsize=16)
    plt.savefig("{}.png".format("_".join(title.split(" "))), bbox_inches="tight")

### Figure 4. Create GHG shares as bar_plot comparison

In [ ]:
def ghg_shares(scenarios, region="all", title="scenario", titles=[], rows=1):
    for model_nm in scenarios:
        fig = plt.figure(figsize=(14, 10))
        mod_list = ["a.", "b.", "c.", "d."]
        for i, scen_nm in enumerate(scenarios[model_nm]):
            # Load scenario
            scen = message_ix.Scenario(mp, model_nm, scen_nm)

            if region == "all":
                regs = get_nodes(scen)
            else:
                regs = region

            # Retrieve list of land scenarios
            land_scen = scen.set("land_scenario")

            # Create X-array, which will contain all the GHG price categories
            Xs, ghg_cat = retrieve_ghgcat(land_scen)

            ## Set position
            cols = int(len(scenarios[model_nm]) / rows)
            index = scenarios[model_nm].index(scen_nm) + 1

            ax = fig.add_subplot(rows, cols, index)

            # Create a dataframe which calcaultes the share of each GHG category used over time.
            # Retrieve the results from the scenario
            df = scen.var("LAND")
            df["ghg_cat"] = df.land_scenario.apply(lambda x: "GHG" + x.split("GHG")[1])
            years = list(set(df.year))
            df = df.drop(["node", "mrg", "land_scenario"], axis=1).set_index(["year"])

            # Retrieve the duration periods of timesteps
            df["duration"] = scen.par(
                "duration_period", filters={"year": years}
            ).set_index("year")["value"]

            # multiply the activity by annual duration
            df["lvl"] *= df["duration"]
            df = (
                df.reset_index()
                .groupby(["ghg_cat"])
                .sum()
                .drop(["duration", "year"], axis=1)
            )
            df["lvl"] /= df["lvl"].sum()
            df["lvl"] *= 100

            # Sort values
            df["GHG"] = ghg_cat["GHG"]
            df = df.sort_values(by="GHG").reset_index()
            ax.barh(df.ghg_cat, df.lvl)

            if title == "scenario":
                ax.set_title(titles[index - 1], fontsize=12)

            if index in [3, 4]:
                ax.set_xlabel("Share of cumulative use\nin [%]", fontsize=14)
            if index in [1, 3]:
                ax.set_ylabel("Carbon price category", fontsize=14)
            ax.set_xticks(list(range(0, 110, 10)))
            ax.tick_params(axis="both", which="major", labelsize=11)
            ax.set_frame_on(True)
            ax.grid(axis="both", which="both", linestyle="--", alpha=0.5)
            ax.text(-5, 13, mod_list[i], fontsize=14)

        if region == "all":
            reg_title = "Global"
        else:
            reg_title = region[0]

        plt.suptitle(
            "{} distribution of carbon price category use\nCumulated from {} to {}".format(
                reg_title, ymin, ymax
            ),
            fontsize=16,
        )

        plt.savefig(
            "{}_{}_validation_cprice.png".format(model_nm, reg_title),
            bbox_inches="tight",
            transparent=True,
        )

### Figure 5. Create Temp/CO2 line plots

In [ ]:
def CDL_Temp_Price():
    # Data comes from model: CD-Links_SSP2_v2
    CO2Price_data = pd.DataFrame(
        {
            "Region": ["World", "World", "World", "World"],
            "Model": [
                "CD-Links_SSP2_v2",
                "CD-Links_SSP2_v2",
                "CD-Links_SSP2_v2",
                "CD-Links_SSP2_v2",
            ],
            "Scenario": ["NPi2020 1000", "NPi2020 1600", "NPi2020 400", "baseline"],
            "Variable": [
                "Price|Carbon",
                "Price|Carbon",
                "Price|Carbon",
                "Price|Carbon",
            ],
            "Unit": [
                "US$2010/t CO2",
                "US$2010/t CO2",
                "US$2010/t CO2",
                "US$2010/t CO2",
            ],
            2000: [0, 0, 0, 0],
            2005: [0, 0, 0, 0],
            2010: [0, 0, 0, 0],
            2020: [2.346, 2.346, 2.346, 0],
            2030: [36.75, 18.846, 120.293, 0],
            2040: [59.861, 30.699, 195.944, 0],
            2050: [97.508, 50.005, 319.173, 0],
            2060: [158.83, 81.453, 519.899, 0],
            2070: [258.717, 132.678, 846.86, 0],
            2080: [421.424, 216.119, 1379.446, 0],
            2090: [686.455, 352.035, 2246.973, 0],
            2100: [1118.162, 573.429, 3660.082, 0],
        }
    )

    Temp_data = pd.DataFrame(
        {
            "Region": ["World", "World", "World", "World"],
            "Model": [
                "CD-Links_SSP2_v2",
                "CD-Links_SSP2_v2",
                "CD-Links_SSP2_v2",
                "CD-Links_SSP2_v2",
            ],
            "Scenario": ["NPi2020 1000", "NPi2020 1600", "NPi2020 400", "baseline"],
            "Variable": [
                "Diagnostics|MAGICC6|Temperature|Global Mean",
                "Diagnostics|MAGICC6|Temperature|Global Mean",
                "Diagnostics|MAGICC6|Temperature|Global Mean",
                "Diagnostics|MAGICC6|Temperature|Global Mean",
            ],
            "Unit": ["°C", "°C", "°C", "°C"],
            2000: [0.798, 0.798, 0.798, 0.798],
            2005: [0.913, 0.913, 0.913, 0.913],
            2010: [0.985, 0.985, 0.985, 0.985],
            2020: [1.227, 1.227, 1.227, 1.227],
            2030: [1.481, 1.47, 1.479, 1.475],
            2040: [1.674, 1.699, 1.619, 1.76],
            2050: [1.777, 1.864, 1.657, 2.052],
            2060: [1.804, 1.96, 1.629, 2.352],
            2070: [1.787, 1.999, 1.564, 2.668],
            2080: [1.75, 2.003, 1.48, 3.008],
            2090: [1.696, 1.979, 1.386, 3.377],
            2100: [1.617, 1.93, 1.282, 3.767],
        }
    )

    fig, axs = plt.subplots(1, 2, figsize=(7, 5))
    leg_args = dict(
        loc="upper right", bbox_to_anchor=(1.5, 1), shadow=True, title="Scenarios"
    )

    # Plot figure 1 - Temp
    ax = axs[0]
    plot_data = pyam.IamDataFrame(Temp_data)
    plot_data.line_plot(ax=ax, color="scenario", legend=False, title=False)
    ax.grid(axis="both", which="both", linestyle="--", alpha=0.5)
    ax.text(x=2000, y=4, s="a.", fontsize=12)
    ax.set_title("Global Temperature\nin [°C]")
    ax.set_xlim([2010, 2100])

    # Plot figure 2 - CO2-Price
    ax = axs[1]
    plot_data = pyam.IamDataFrame(CO2Price_data)
    plot_data.line_plot(ax=ax, color="scenario", title=False)
    ax.grid(axis="both", which="both", linestyle="--", alpha=0.5)
    ax.text(x=2000, y=4000, s="b.", fontsize=12)
    ax.set_title("Global Carbon price\nin [US$2010/t CO2]")
    ax.set_xlim([2010, 2100])

    # General figure layout
    ax.legend(**leg_args)
    plt.subplots_adjust(right=1.2)
    plt.savefig(
        "CD_Links_SSP2_v2_Global_Cprice_Temp.png", bbox_inches="tight", transparent=True
    )

In [ ]:
def SSP1_Feedback():
    # Data comes from model: MESSAGEV SSP1 scenarios
    ghg_data = pd.DataFrame(
        {
            "Region": ["World"] * 10,
            "Model": ["SSP1"] * 10,
            "Scenario": [
                "Baseline",
                "RCP6.0",
                "RCP4.5",
                "RCP3.4",
                "RCP2.6",
                "Baseline_fb",
                "RCP6.0_fb",
                "RCP4.5_fb",
                "RCP3.4_fb",
                "RCP2.6_fb",
            ],
            "Variable": ["AFOLU GHG Emissions"] * 10,
            "Unit": ["Mt CO2e/yr"] * 10,
            2005: [
                6896.144392,
                6896.144392,
                6896.144392,
                6896.144392,
                6896.144392,
                6902.368774,
                6902.368774,
                6902.368774,
                6902.368774,
                6902.313774,
            ],
            2010: [
                7069.24612,
                7069.24612,
                7069.24612,
                7069.24612,
                7069.24612,
                7053.06512,
                7053.06512,
                7053.06512,
                7053.06512,
                7052.95512,
            ],
            2020: [
                6098.686593,
                6098.681691,
                6069.944879,
                4452.197659,
                3546.215527,
                6015.902864,
                5992.560771,
                5478.837467,
                4773.387581,
                3669.284766,
            ],
            2030: [
                4781.615072,
                4781.511171,
                3998.303125,
                975.3400152,
                -231.697102,
                4415.807263,
                4319.401096,
                3064.475192,
                941.178431,
                -286.256831,
            ],
            2040: [
                3877.474062,
                3876.311445,
                1625.422377,
                -510.5914792,
                -1300.749902,
                3472.011917,
                3163.256871,
                698.376577,
                -890.446184,
                -1713.948375,
            ],
            2050: [
                2317.818117,
                2317.563306,
                -334.4759677,
                -1630.695697,
                -2373.340516,
                2114.346675,
                1934.612868,
                -1086.6529,
                -1967.498431,
                -2582.508434,
            ],
            2060: [
                969.0417443,
                968.9525377,
                -2001.991083,
                -3080.200312,
                -3771.046326,
                878.013049,
                640.117504,
                -2541.201192,
                -3083.358215,
                -3587.242856,
            ],
            2070: [
                -105.6330768,
                -106.5546605,
                -2984.18353,
                -3765.119679,
                -4209.407759,
                -307.838512,
                -674.432632,
                -3571.263594,
                -4067.537049,
                -3955.179902,
            ],
            2080: [
                -817.1243679,
                -812.984645,
                -3589.393333,
                -4325.383025,
                -4670.600923,
                -993.052909,
                -1507.688012,
                -4259.195631,
                -4219.559666,
                -4384.855758,
            ],
            2090: [
                -1295.987509,
                -1292.704935,
                -4222.90096,
                -4825.659287,
                -5512.273113,
                -1073.645061,
                -1857.264878,
                -4407.972032,
                -4718.843039,
                -4221.234495,
            ],
            2100: [
                -1622.617031,
                -1619.383046,
                -4196.417841,
                -4556.093054,
                -5077.268216,
                -1518.686825,
                -2337.357657,
                -4258.905929,
                -4368.817301,
                -3802.122395,
            ],
        }
    )

    ch4_data = pd.DataFrame(
        {
            "Region": ["World"] * 10,
            "Model": ["SSP1"] * 10,
            "Scenario": [
                "Baseline",
                "RCP6.0",
                "RCP4.5",
                "RCP3.4",
                "RCP2.6",
                "Baseline_fb",
                "RCP6.0_fb",
                "RCP4.5_fb",
                "RCP3.4_fb",
                "RCP2.6_fb",
            ],
            "Variable": ["AFOLU CH4 Emissions"] * 10,
            "Unit": ["Mt CH4/yr"] * 10,
            2005: [
                122.6597143,
                122.6597143,
                122.6597143,
                122.6597143,
                122.6597143,
                122.239,
                122.239,
                122.239,
                122.239,
                122.239,
            ],
            2010: [
                125.307,
                125.307,
                125.307,
                125.307,
                125.307,
                125.307,
                125.307,
                125.307,
                125.307,
                125.307,
            ],
            2020: [
                131.0504762,
                131.0505714,
                130.6721429,
                120.7557619,
                112.4821905,
                131.045,
                130.974,
                128.112,
                122.323,
                116.764,
            ],
            2030: [
                134.5657619,
                134.5658571,
                129.2542857,
                117.7830952,
                108.2467619,
                134.632,
                134.334,
                127.527,
                118.55,
                111.33,
            ],
            2040: [
                135.4065714,
                135.4050476,
                124.2382381,
                114.4189524,
                103.8475714,
                135.482,
                135.19,
                123.48,
                113.539,
                104.654,
            ],
            2050: [
                134.3568095,
                134.3560476,
                118.9612381,
                108.8656667,
                97.72742857,
                134.463,
                133.987,
                118.476,
                107.438,
                97.31,
            ],
            2060: [
                131.3710952,
                131.3709048,
                112.5431905,
                101.7245714,
                90.24404762,
                131.613,
                130.898,
                110.963,
                99.52,
                89.822,
            ],
            2070: [
                127.204,
                127.2121905,
                105.0195238,
                93.42195238,
                82.84647619,
                127.388,
                126.207,
                102.799,
                90.571,
                81.078,
            ],
            2080: [
                122.5970476,
                122.5814286,
                97.20609524,
                85.53,
                75.25847619,
                122.342,
                120.428,
                93.151,
                82.112,
                72.397,
            ],
            2090: [
                116.3804286,
                116.3734286,
                89.28795238,
                77.82280952,
                66.25742857,
                115.896,
                111.811,
                83.399,
                73.015,
                64.302,
            ],
            2100: [
                109.6945714,
                109.5599048,
                80.69219048,
                69.0387619,
                58.93404762,
                109.38,
                103.774,
                74.25,
                64.294,
                56.75,
            ],
        }
    )

    fig, axs = plt.subplots(1, 2, figsize=(7, 5))
    leg_args = dict(
        loc="upper right", bbox_to_anchor=(1.5, 1), shadow=True, title="Scenarios"
    )

    # Plot figure 1 - AFOLU GHG Emissions
    ax = axs[0]
    ghg_data = ghg_data[
        ghg_data["Scenario"].isin(
            ["Baseline", "RCP4.5", "RCP2.6", "Baseline_fb", "RCP4.5_fb", "RCP2.6_fb"]
        )
    ]
    for typ in ["emulator", "feedback"]:
        if typ == "emulator":
            ghg_data_flt = ghg_data[ghg_data["Scenario"].str.find("fb") < 0]
            linestyle = "-"
        else:
            ghg_data_flt = ghg_data[ghg_data["Scenario"].str.find("fb") >= 0]
            linestyle = ":"
        plot_data = pyam.IamDataFrame(ghg_data_flt)
        plot_data.line_plot(
            ax=ax, color="scenario", legend=False, title=False, linestyle=linestyle
        )
    ax.grid(axis="both", which="both", linestyle="--", alpha=0.5)
    ax.text(x=2000, y=8500, s="a.", fontsize=12)
    ax.set_title("AFOLU GHG Emissions\nin [Mt CO2e/yr]")
    ax.set_xlim([2010, 2100])

    # Plot figure 2 - AFOLU CH4 Emissions
    ax = axs[1]
    ch4_data = ch4_data[
        ch4_data["Scenario"].isin(
            ["Baseline", "RCP4.5", "RCP2.6", "Baseline_fb", "RCP4.5_fb", "RCP2.6_fb"]
        )
    ]
    for typ in ["emulator", "feedback"]:
        if typ == "emulator":
            ch4_data_flt = ch4_data[ch4_data["Scenario"].str.find("fb") < 0]
            linestyle = "-"
        else:
            ch4_data_flt = ch4_data[ch4_data["Scenario"].str.find("fb") >= 0]
            linestyle = ":"
        plot_data = pyam.IamDataFrame(ch4_data_flt)
        plot_data.line_plot(
            ax=ax, color="scenario", legend=False, title=False, linestyle=linestyle
        )
    ax.grid(axis="both", which="both", linestyle="--", alpha=0.5)
    ax.text(x=2000, y=145, s="b.", fontsize=12)
    ax.set_title("AFOLU CH4 Emissions\nin [Mt CH4/yr]")
    ax.set_xlim([2010, 2100])

    leg1_args = dict(
        loc="upper right", bbox_to_anchor=(1.5, 1), shadow=True, title="Scenarios"
    )
    handles, labels = ax.get_legend_handles_labels()
    leg1 = ax.legend(
        [handles[:3][i] for i in [0, 2, 1]],
        [labels[:3][i] for i in [0, 2, 1]],
        **leg1_args,
    )

    leg2_args = dict(
        loc="upper right", bbox_to_anchor=(1.51, 0.7), shadow=True, title="Run type"
    )
    leg2 = ax.legend([handles[0], handles[3]], ["emulator", "feedback"], **leg2_args)
    ax.add_artist(leg1)

    plt.subplots_adjust(right=1.2)
    plt.savefig("SSP1_Feedback.png", bbox_inches="tight", transparent=True)

## Call plotting routines

In [ ]:
# Set range of years for which data should be cumulated
ymin = 2010
ymax = 2100

# Choose which figures to make
Fig1_LU_matrix = 0
Fig2_LU_developments = 0
Fig3_LU_surface = 0
F3_plain = 0
F3_CDL = 0
F3_Price = 0
Fig4_GHG_shares = 0
Fig5_CDL_Temp_Price = 0
Fig6_SSP1_Feedback = 0

In [ ]:
if Fig1_LU_matrix:
    scenarios = ["CD_Links_SSP2_v2", "baseline"]
    scen = message_ix.Scenario(mp, scenarios[0], scenarios[1])
    lu_matrix(scen)

if Fig2_LU_developments:
    scenarios = ["CD_Links_SSP2_v2", "baseline"]
    scen = message_ix.Scenario(mp, scenarios[0], scenarios[1])
    region = "R11_LAM"
    land_scen = ["BIO00GHG000", "BIO00GHG2000", "BIO06GHG000", "BIO06GHG2000"]
    lu_developments(scen, region, land_scen)

if Fig3_LU_surface:
    if F3_plain:
        # Plot "plain" baseline trade-off surface
        scenarios = {"CD_Links_SSP2_v2": ["baseline"]}
        lu_surface(scenarios)

    if F3_CDL:
        # Plot scenario surface with results indicated as part of  surface
        scenarios = {
            "CD_Links_SSP2_v2": [
                "baseline",
                "NPi2020_1600-con-prim-dir-ncr",
                "NPi2020_1000-con-prim-dir-ncr",
                "NPi2020_400-con-prim-dir-ncr",
            ]
        }

        titles = ["Baseline", "NPi 1600", "NPi 1000", "NPi 400"]
        lu_surface(
            scenarios, plot_results=True, title="scenario", titles=titles, rows=2
        )

if Fig4_GHG_shares:
    # Plot scenario surface with results indicated as part of  surface
    scenarios = {
        "ENGAGE_SSP2_v4.1.2_sens": [
            "baseline_CPrice_const_5",
            "baseline_CPrice_const_100",
            "baseline_CPrice_const_400",
            "baseline_CPrice_const_1000",
        ]
    }
    titles = [
        "GHG005\nCarbon Price at {}$2010/tCO2".format(round(5 * usd_conv_2000_2010, 2)),
        "GHG100\nCarbon Price at {}$2010/tCO2".format(
            round(100 * usd_conv_2000_2010, 2)
        ),
        "GHG400\nCarbon Price at {}$2010/tCO2".format(
            round(400 * usd_conv_2000_2010, 2)
        ),
        "GHG1000\nCarbon Price at {}$2010/tCO2".format(
            round(1000 * usd_conv_2000_2010, 2)
        ),
    ]

    ghg_shares(scenarios, titles=titles, rows=2)

if Fig5_CDL_Temp_Price:
    CDL_Temp_Price()

if Fig6_SSP1_Feedback:
    SSP1_Feedback()